In [1]:
import duckdb
import dotenv

In [2]:
env = dotenv.dotenv_values(".env")

In [3]:
env

OrderedDict([('KITE_API_KEY', 'dvrupglqqvbe45r5'),
             ('KITE_API_SECRET', '892acup2e35yb198d6lpqjhwawn90dgj'),
             ('KITE_CLIENT_ID', 'ZE1919'),
             ('R2_ACCOUNT_ID', 'bb915458af20c94d1e79419ea3b78828'),
             ('R2_ACCESS_KEY_ID', 'c7fe1c24cad4497076a96f70023184e2'),
             ('R2_SECRET_ACCESS_KEY',
              '25b9a32dd30e2f8b4473ad0e23cd405aafc5018d2bc304fa2923a6146b2118df'),
             ('KITE_ACCESS_TOKEN', 'Ikf8qqvlGMARr2gTqH3O1YAy7vyWdqXe'),
             ('ENVIRONMENT', 'development'),
             ('DEBUG', 'false'),
             ('SAMPLE_MODE', 'false'),
             ('SAMPLE_SIZE', '1000'),
             ('AMFI_NAV_TIMEOUT', '30'),
             ('AMFI_SCHEME_TIMEOUT', '30'),
             ('MAX_RETRIES', '3'),
             ('RETRY_DELAY', '5'),
             ('HISTORICAL_FETCH_DAYS', '90'),
             ('HISTORICAL_BATCH_SIZE', '15'),
             ('CHUNK_SIZE', '10000'),
             ('MAX_MEMORY_GB', '2.0'),
             ('PARQUET_CO

In [4]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [5]:
con = duckdb.connect()

In [6]:
con.install_extension('httpfs')
con.load_extension('httpfs')

In [7]:
con.sql(f"""
CREATE OR REPLACE SECRET (
    TYPE r2,
    KEY_ID '{env['R2_ACCESS_KEY_ID']}',
    SECRET '{env['R2_SECRET_ACCESS_KEY']}',
    ACCOUNT_ID '{env['R2_ACCOUNT_ID']}'
);
""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [8]:
test = con.read_parquet("r2://financial-data-store/clean/commodities_bhav_clean.parquet")

In [11]:
test.limit(5)

┌────────────┬───────────┬─────────────────────┬─────────────────────┬────────┬────────┬────────┬────────┬────────┬───────────────────┬────────┬──────────┐
│ instrument │  symbol   │      expiry_dt      │        date         │  open  │  high  │  low   │ close  │ volume │ volumeinthousands │ value  │ open_int │
│  varchar   │  varchar  │    timestamp_ns     │    timestamp_ns     │ double │ double │ double │ double │ int64  │      varchar      │ double │  int64   │
├────────────┼───────────┼─────────────────────┼─────────────────────┼────────┼────────┼────────┼────────┼────────┼───────────────────┼────────┼──────────┤
│ FUTCOM     │ ALUMINIUM │ 2023-01-31 00:00:00 │ 2023-01-02 00:00:00 │ 208.85 │  209.7 │  208.5 │ 209.55 │    839 │ 4195.000 KGS      │ 8773.9 │     4033 │
│ FUTCOM     │ ALUMINIUM │ 2023-02-28 00:00:00 │ 2023-01-02 00:00:00 │  210.0 │ 211.35 │  210.0 │  211.1 │     54 │ 270.000 KGS       │  568.6 │       90 │
│ FUTCOM     │ ALUMINIUM │ 2023-03-31 00:00:00 │ 2023-01-02 00:0

In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
def clean_cols(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ','_').str.replace('.','_').str.replace('(','').str.replace(')','')
    return df
    
    

In [ ]:
col_select = ['Scheme Code', 'ISIN Div Payout/ISIN Growth', 'ISIN Div Reinvestment', 'Net Asset Value', 'Date']

In [ ]:
raw_csvs = pd.concat([pd.read_csv(f)[col_select] for f in Path('/home/modiis/projects/mf_data_pipeline/data/raw/nav_historical').glob('*.csv')], ignore_index=True)

In [ ]:
raw_csvs.head()

In [ ]:
raw_csvs.shape

In [ ]:
raw_csvs.head()

In [ ]:
clean_df = (raw_csvs
         .rename(columns={
            'Scheme Code': 'scheme_code',
            'ISIN Div Payout/ISIN Growth': 'isin_growth', 
            'ISIN Div Reinvestment': 'isin_dividend',
            'Net Asset Value': 'nav',
            'Date': 'date'
        })
         .query('scheme_code.notnull() & nav.notnull() & date.notnull()')
         .assign(scheme_code=lambda x: x['scheme_code'].astype(str),
                 date = lambda x: pd.to_datetime(x['date'], format='%d-%b-%Y', errors='coerce'),
                 nav = lambda x: pd.to_numeric(x['nav'], errors='coerce'),         
        )
        )

In [ ]:
clean_df.head()

In [ ]:
from config.settings import R2

In [ ]:
r2 = R2()

In [ ]:
path = r2.get_full_path('raw','nav_historical')

In [ ]:
path

In [ ]:
con.write_parquet(clean_df, path, overwrite=True)

In [ ]:
FILE_PATH

In [ ]:
con.read_parquet(r2)

In [ ]:
# Register your DataFrame with DuckDB
con.register('clean_df_table', clean_df)

In [ ]:
con.execute(f"""
    COPY clean_df_table TO '{path}' 
    (FORMAT PARQUET)
""")

In [ ]:
con.read_parquet(pat)